#Correlation matricies - 28/10/19

In this script, the preamble is most important, rather than the machine learning. We are looking at correlated variables to see which ones would be the best to try to recreate. 

<font color='orange'>16/11/19 - It has been brought to our attention that the S1 and S2 variables have not been accurately reproduced in this dataset. However the layering may still work?? The correlation matricies in this are now no longer useful. It has also come to our attention that these need not be recreated. Since making this script we have decided the focus should be on reproducing S1, S2 and f200 </font>

In [4]:
pip install uproot

In [5]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


In [11]:
cd gdrive/'My Drive'/Masters_Project/test_run

/content/gdrive/My Drive/Masters_Project/test_run


Import relevant libraries & use uproot to open it.

In [0]:
import uproot
import pandas
import numpy as np

tree = uproot.open("dark_matter_100kev.root")["dstree"]

The different variables contained within the tree.

In [13]:
tree.show()

ev                         (no streamer)              asdtype('>i4')
pdg                        (no streamer)              asdtype('>i4')
ene0                       (no streamer)              asdtype('>f4')
s1ene                      (no streamer)              asdtype('>f4')
s2ene                      (no streamer)              asdtype('>f4')
veto_visene                (no streamer)              asdtype('>f4')
mu_visene                  (no streamer)              asdtype('>f4')
vetoene                    (no streamer)              asdtype('>f4')
muene                      (no streamer)              asdtype('>f4')
tpcene                     (no streamer)              asdtype('>f4')
x                          (no streamer)              asdtype('>f4')
y                          (no streamer)              asdtype('>f4')
z                          (no streamer)              asdtype('>f4')
ene                        (no streamer)              asdtype('>f4')
r                          (no str

Training Dataset is the x variable.

In [14]:
pip install physt

     |████████████████████████████████| 92kB 5.4MB/s 
  Created wheel for physt: filename=physt-0.4.8.3-cp36-none-any.whl size=96728 sha256=394e232e868edf00767521764251eb476cb6ef2cf9077312f4205a25df2a9669
  Stored in directory: /root/.cache/pip/wheels/9c/fd/76/18f1e6c1462bb8370086639ae169f314f5f5ddf2b9abf0dcd8
Successfully built physt


In [20]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from physt import h1, h2, histogramdd
import matplotlib.cm as cm

x = tree.array("dep_x")
y = tree.array("dep_y")
en = tree.array("dep_ene")

i=5
#for i in range(0,len(x)):
histogram = h2(x[i], en[i], "fixed_width", (5, 200),name="energy vs x", axis_names=["x", "energy"])
histogram2 = h2(y[i], en[i], "fixed_width", (5, 200), name="energy vs y", axis_names=["y", "energy"])
histogram3 = h2(x[i], y[i], "fixed_width", (5, 5), name="x vs y", axis_names=["x", "y"])


fig, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()

ax1 = histogram.plot(ax=ax1, cmap=cm.copper,grid_color=cm.copper(0.5))
ax2 = histogram2.plot(ax=ax2, cmap=cm.copper, grid_color=cm.copper(0.5))
ax3 = histogram3.plot(ax=ax3, cmap=cm.copper, grid_color=cm.copper(0.5))

'''
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x,y)
plt.show()
'''

ValueError: ignored

In [0]:
import seaborn as sns

pandasDS=tree.pandas.df(["ene0","s1ene","s2ene","x","y","z","px","py","pz","tpcene","npeS1","npeS2"],flatten=False)

x = tree.array("dep_x")
y = tree.array("dep_y")

meanDepX=[]
meanDepY=[]

for i in range(0,len(x)):
  meanDepX.append(np.mean(x[i]))
  meanDepY.append(np.mean(y[i]))

pandasDS["meanDepX"]=meanDepX
pandasDS["meanDepY"]=meanDepY

print(pandasDS)
corr = pandasDS.corr()
sns.heatmap(corr, 
            center=0,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [0]:
var = "s2ene"
train_DS = tree.array(f"{var}") / 8000

In [0]:
plt.hist(train_DS, density = True)

In [0]:
len(train_DS)

In [0]:
num_train = int(1e4)
noise_size = 500

In [0]:
#  Model dependent constants
#  Define model PDF

#The range of the model
range_min, range_max = 0, 1
#The range being plot
range_width = range_max - range_min

plot_resolution = 200

#Initial weights
w1, m1, s1 = 0.5, 8., 1.
w2, m2, s2 = 0.5, 14, 5.

In [0]:
#  Create GAN

from tensorflow.keras.layers     import BatchNormalization, Dense, Dropout, Input, LeakyReLU
from tensorflow.keras.models     import Model, Sequential
from tensorflow.keras.optimizers import Adam

#Discriminator
#See how number of neurons are sae for each layer to reduce number of hyperparameters
d = Sequential(name="Discriminator")
d.add(Dense(25, activation="relu", input_shape=(1,)))
#d.add(Dense(50, activation="relu"))
#d.add(Dense(50))
#d.add(Dense(50, activation="relu"))
#d.add(Dense(50))
d.add(LeakyReLU(0.2))
d.add(Dense(2, activation="softmax"))
d.compile(loss="categorical_crossentropy", optimizer=Adam(0.0002, beta_1=0.9), metrics=["accuracy"])
d.summary()

g = Sequential(name="Generator")
g.add(Dense(25, activation="relu", input_shape=(noise_size,)))
g.add(BatchNormalization())
g.add(Dense(25, activation="relu"))
g.add(BatchNormalization())
#g.add(Dense(50, activation="relu"))
#g.add(BatchNormalization())
#g.add(Dense(50, activation="relu"))
#g.add(BatchNormalization())
g.add(Dense(1, activation="linear"))
g.summary()

input_layer = Input((noise_size,))
output_layer = d(g(input_layer))
gan = Model(input_layer, output_layer)
d.trainable = False
gan.compile(loss="categorical_crossentropy", optimizer=Adam(0.0002, beta_1=0.5), metrics=["accuracy"])
gan.summary()

In [0]:
#  Func: plot GAN progress
def plot_points_GAN (data, label, epoch, batch_size, d_acc, gan_acc) :
    
    #y = plt.hist(train_DS , alpha = 0.3,density = True, label = 'Real Data')
    #x = np.linspace(range_min, range_max, 200)
    h = plt.hist(data, bins=np.linspace(range_min, range_max, 1+int(range_width/plot_resolution)), color="blue", alpha=0.6,density = True, label=label)
    plt.legend(loc="upper right", fontsize=14)
    plt.xlabel("x", size=14, labelpad=10)
    #y_max = 1.5*max([max(y),max(h[0])])
    y_max = 8
    plt.ylabel(r"$\rho\left(x\right)$", size=14, labelpad=30, rotation="horizontal")
    plt.gca().set_xlim([range_min-0.1*range_width, range_max+0.1*range_width])
    '''plt.text(range_min+0.*1.1*range_width   , 0.84*y_max, "GAN test", size=18, style="italic", weight="bold")
    plt.text(range_min+0.750*1.1*range_width, 0.68*y_max, f"Epoch: {epoch}")
    plt.text(range_min+0.683*1.1*range_width, 0.61*y_max, f"Batch size: {batch_size}")
    plt.text(range_min+0.780*1.1*range_width, 0.54*y_max, r"$\epsilon_{\rm disc}$: "+f"{int(100.*d_acc)}%")
    plt.text(range_min+0.780*(range_max-range_min), 0.47*y_max, r"$\epsilon_{\rm GAN}$: "+f"{int(100.*gan_acc)}%")
    '''
    plt.subplots_adjust(left=0.18, right=0.98, top=0.95, bottom=0.15)
    plt.savefig("Train_epoch{epoch}.png", dpi=96)
    plt.show()

In [0]:
#  Train GAN

epochs     = 5000
batch_size = 100

def update_batch_size (gen_DS) :
    global batch_size
    gen_median, gen_std = np.median(gen_DS.flatten()), np.std(gen_DS.flatten())
    if np.fabs((gen_median-train_median)/train_std) > 1./np.sqrt(batch_size) : return
    if np.fabs((gen_std/train_std)-1) > 1./np.sqrt(batch_size) : return
    '''if batch_size > 200 :
        gen_quantiles = np.quantile(gen_DS, quantiles) - gen_median
        for gen_q, train_q in zip(gen_quantiles, train_quantiles) :
            if np.fabs((gen_q/train_q)-1) > 1./np.sqrt(batch_size) : return'''
    batch_size = min(100+batch_size, num_train)
    
    epochs_saved = []



In [0]:
epochs_saved = []
loss = []
gen_loss = []
gen_acc = []
real_loss = []
real_acc = []
generator_loss = []
epoch_no = []
for e in range(epochs) :
    noise  = np.random.uniform(low = 0, high = 1, size=(batch_size, noise_size))
    
    #Get the real and fake data
    batch_DS = train_DS[np.random.randint(0, len(train_DS), batch_size)].reshape(batch_size, 1)
    #fake data made from predicting noise
    #print(batch_DS)
    gen_DS   = g.predict(noise)
    
    #Real data has label [1.,0]
    real_label = np.array([[1., 0.] for i in range(batch_size)])
    
    #Fake data has label [0.,1.]
    fake_label = np.array([[0., 1.] for i in range(batch_size)])
    
    #Combine real and generated data
    X = np.concatenate([batch_DS  , gen_DS    ])
    Y = np.concatenate([real_label, fake_label])
    
    
    d.trainable = True
    d_loss, d_acc = d.train_on_batch(X, Y)
        
    noise  = np.random.uniform(low = 0, high = 1, size=(batch_size, noise_size))
    d.trainable = False
    #gan.train_on_batch(noise, real_label)
    
    if e == 0 or (e+1) % 100 == 0 :
        noise  = np.random.uniform(low = 0, high = 1, size=(batch_size, noise_size))
        gen_DS = g.predict(noise)
        print(d.evaluate(X, Y, verbose=0))
        epoch_no.append(e+1)
        plt.hist(train_DS, alpha = 0.3,density = True, range = (0,0.3), bins = 10, label = 'Real Data')
        plt.hist(gen_DS, density = True, range = (0,0.3), bins = 10, label = 'Gen Data')
        plt.legend(loc="upper right", fontsize=10)
        plt.xlabel(f"{var}", size=14, labelpad=10)
        plt.ylabel(r"$\rho\left(x\right)$", size=14, labelpad=30, rotation="horizontal")
        plt.title(f"Data for Batch Size:{batch_size} and Epoch {e+1}")
        plt.show()
        real_loss.append(d_loss)
        real_acc.append(d_acc)
        #gen_loss.append(d1_loss)
        #gen_acc.append(d1_acc)
        epochs_saved.append(e+1)


Plotting Discriminator Accuracy

In [0]:
del epoch_no[-1]
h3 = plt.plot(epoch_no,real_loss, color = 'blue', label = "Loss")
h4 = plt.plot(epoch_no,real_acc, color = 'orange', label = "Accuracy")
plt.xlabel("Epoch number", size=14, labelpad=10)
plt.ylabel("Accuracy/Loss", size=14, labelpad=30, rotation="vertical")
plt.title(f"Disciminator variables with batch size {batch_size} for {var}")
plt.legend(loc="upper right", fontsize=10)
plt.show()

In [0]:
batch_size=500
noise  = np.random.uniform(low = 0, high = 1, size=(batch_size, noise_size))
gen_DS = g.predict(noise)
print(d.evaluate(X, Y, verbose=0))
epoch_no.append(e)
plt.hist(train_DS, alpha = 0.3,bins = 10,density = True, label = 'Real Data')
plt.hist(gen_DS,bins = 10, density = True, label = 'Gen Data')
plt.show()